In [1]:
!nvidia-smi

Tue Nov 25 11:19:04 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   72C    P0             148W / 300W |  10340MiB / 81920MiB |     70%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach1


In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
import re
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast
import os
from tqdm import tqdm

# 1. define punctuation map
punctuation_map = {
    ',': 'COMMA',
    '.': 'PERIOD',
    '?': 'QUESTION',
    '!': 'EXCLAMATION',
    ';': 'SEMICOLON',
    ':': 'COLON',
    '-': 'HYPHEN',
    '–': 'EN_DASH',
    '—': 'EM_DASH',
    '(': 'LEFT_PAREN',
    ')': 'RIGHT_PAREN',
    '[': 'LEFT_BRACKET',
    ']': 'RIGHT_BRACKET',
    '{': 'LEFT_BRACE',
    '}': 'RIGHT_BRACE',
    '"': 'DOUBLE_QUOTE',
    "'": 'SINGLE_QUOTE',
    '…': 'ELLIPSIS',
    '/': 'SLASH',
    '\\': 'BACKSLASH',
    '@': 'AT_SYMBOL',
    '#': 'HASH',
    '$': 'DOLLAR',
    '%': 'PERCENT',
    '&': 'AMPERSAND',
    '*': 'ASTERISK',
    '+': 'PLUS',
    '=': 'EQUALS',
    '<': 'LESS_THAN',
    '>': 'GREATER_THAN',
    '|': 'PIPE',
    '^': 'CARET',
    '`': 'BACKTICK',
    '~': 'TILDE'
}

# Automatically create label_list from punctuation_map
label_list = ["O"] + list(punctuation_map.values())
label_to_id = {l: i for i, l in enumerate(label_list)}

print("Label list:", label_list)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Label list: ['O', 'COMMA', 'PERIOD', 'QUESTION', 'EXCLAMATION', 'SEMICOLON', 'COLON', 'HYPHEN', 'EN_DASH', 'EM_DASH', 'LEFT_PAREN', 'RIGHT_PAREN', 'LEFT_BRACKET', 'RIGHT_BRACKET', 'LEFT_BRACE', 'RIGHT_BRACE', 'DOUBLE_QUOTE', 'SINGLE_QUOTE', 'ELLIPSIS', 'SLASH', 'BACKSLASH', 'AT_SYMBOL', 'HASH', 'DOLLAR', 'PERCENT', 'AMPERSAND', 'ASTERISK', 'PLUS', 'EQUALS', 'LESS_THAN', 'GREATER_THAN', 'PIPE', 'CARET', 'BACKTICK', 'TILDE']


In [6]:
punctuation_reverse_map = {v: k for k, v in punctuation_map.items()}
punctuation_reverse_map["O"] = ""   # no punctuation

In [7]:
from datasets import load_dataset
from evaluate import load
raw_datasets = load_dataset("thenlpresearcher/test_data_marathi")
metric = load("sacrebleu")

2025-11-25 11:19:45.053120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 11:19:45.118255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 11:19:47.642366: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [24]:
import torch

def restore_punctuation(text: str, model, tokenizer, label_list, device, punctuation_reverse_map) -> str:
    """
    Restores punctuation to an unpunctuated text string, utilizing tokenizer subwords
    and word_ids() for accurate mapping.
    """
    # 1. Tokenize the input text
    words = re.findall(r"\w+|[^\w\s]", text.strip())
#     print(words)
    encoded_input = tokenizer(
        words, 
        is_split_into_words=True, 
        return_tensors="pt", 
        padding=True, 
        truncation=True
    ).to(device)
    
    # 2. Extract word IDs and perform inference
    word_ids_list = encoded_input.word_ids() # Maps subword token index to original word index (or None for special tokens)
    tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
    tokens = tokens[1:-1]
    # Run model inference to get logits/predictions
    with torch.no_grad():
        outputs = model(**encoded_input)
        logits = outputs.logits
    
    # Get the predicted label index (p_id) for each token
    # We take the first element [0] because the input is a batch of size 1
    p_ids = torch.argmax(logits, dim=-1).squeeze().tolist()
    
    p_ids = p_ids[1:-1]
    word_ids_list = word_ids_list[1:-1]
#     print(word_ids_list)
#     print(p_ids)
#     print(tokens)
    
    final_output = []
    
    i = 0
    for i in range(len(tokens)):
        t = tokens[i]
        p = p_ids[i]
        punct = punctuation_reverse_map[label_list[p]]
        
        t = t.strip('#')
        if t == punct:
            continue
        
        if i < len(tokens) -1 and punct == tokens[i+1]:
            final_output.extend([t, punct])
            i = i + 2
            continue
            
        if punct != " " and i < len(word_ids_list) - 1 and word_ids_list[i] != word_ids_list[i+1]:
            punct = punct + " "
        if punct == " " and i < len(word_ids_list) - 1 and word_ids_list[i] == word_ids_list[i+1]:
            punct = ""
        
        
        if i < len(word_ids_list) - 1 and word_ids_list[i] == word_ids_list[i+1] and p_ids[i] == p_ids[i+1]:
            final_output.append(t)
        else:
            final_output.extend([t, punct])
        i = i + 1
        
    # 6. Final cleanup
    result = "".join(final_output).strip()
    
    if result:
        # Capitalize the first letter
        return result[0].upper() + result[1:]
    return ""

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer

tokenizer = AutoTokenizer.from_pretrained("thenlpresearcher/bert_punct_model")
model = AutoModelForTokenClassification.from_pretrained("thenlpresearcher/bert_punct_model").to(device)

sentence = "i am going to school but i forgot my bag"
print(restore_punctuation(sentence, model, tokenizer, label_list, device, punctuation_reverse_map))

I am going to school, but i forgot my bag.


In [10]:
raw_datasets["test"][0]

{'Unnamed: 0': 0,
 'punct_type': 'Comma',
 'sent_written': 'Chanting the choir raised the volume as the celebrant intoned the prayer.',
 'sent_meant': 'Chanting, the choir raised the volume as the celebrant intoned the prayer.',
 'gt_marathi': 'जल्लोष करणाऱ्या व्यक्तीने प्रार्थनेचा उच्चार करताच, गायनाने आवाज वाढवला.',
 'gemini_out': 'धर्मगुरू प्रार्थना म्हणत असताना, घोष करणाऱ्या गायकवृंदाने आवाज वाढवला.',
 'cfilt_out': 'उत्सवी प्रार्थनेचा उच्चार करत असताना, जप करत, गायकवृंदाने आवाज वाढवला.'}

In [26]:
predicted_sentences = []

for text in raw_datasets["test"]['sent_written']:
    pred = restore_punctuation(text.strip('.').strip('?'), model, tokenizer, label_list, device, punctuation_reverse_map)
    predicted_sentences.append(pred)
    print(text)
    print(pred)
    print('---')

Chanting the choir raised the volume as the celebrant intoned the prayer.
Chanting: the choir raised the volume as the celebrant intoned the prayer.
---
A six-month-old calf was submitted for examination, showing lameness in all four legs which had been present since soon after birth.
A six-month-old calf was submitted for examination , showing lameness in all four legs, which had been present since soon after birth.
---
Planning authorities should provide alternative locations for small businesses which are or would be offensive in a residential area.
Planning authorities should provide alternative locations for small businesses which are, or would be, offensive in a residential area.
---
As the machine develops the forms we use to record data from past projects will be amended.
As the machine develops, the forms we use to record data from past projects will be amended.
---
As mentioned, first impressions can be misleading.
As mentioned,, first impressions can be misleading.
---
To ge

In [27]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]

        # Preprocess the batch
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )
        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=5,  # ensure some minimum length
                max_length=max_length,
                num_beams=5,
                num_return_sequences=1,
                early_stopping=True,
                decoder_start_token_id=model.config.decoder_start_token_id
            )

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess translations (remove language prefix, entity replacement, etc.)
        translations += ip.postprocess_batch(decoded_texts, lang=tgt_lang)

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [28]:
def initialize_model_and_tokenizer(ckpt_dir, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.eval()

    return tokenizer, model

In [29]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]

        # Preprocess the batch
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )
        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=5,  # ensure some minimum length
                max_length=max_length,
                num_beams=5,
                num_return_sequences=1,
                early_stopping=True,
                decoder_start_token_id=model.config.decoder_start_token_id
            )

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Postprocess translations (remove language prefix, entity replacement, etc.)
        translations += ip.postprocess_batch(decoded_texts, lang=tgt_lang)

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [30]:
import torch
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [31]:
model = en_indic_model.to(DEVICE)

In [32]:
tokenizer = en_indic_tokenizer

In [33]:
raw_datasets['test']

Dataset({
    features: ['Unnamed: 0', 'punct_type', 'sent_written', 'sent_meant', 'gt_marathi', 'gemini_out', 'cfilt_out'],
    num_rows: 54
})

In [34]:
from IndicTransToolkit.processor import IndicProcessor

In [35]:
ip = IndicProcessor(inference=True)

In [36]:
BATCH_SIZE = 4
src_lang, tgt_lang = "eng_Latn", "mar_Deva"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------- LOAD DATA --------------------
src_sentences = predicted_sentences
ref_gt     = raw_datasets['test']["gt_marathi"]
ref_gem    = raw_datasets['test']["gemini_out"]
ref_cfilt  = raw_datasets['test']["cfilt_out"]

In [37]:
print(len(src_sentences))
print(len(ref_gt))
print(len(ref_gem))
print(len(ref_cfilt))

54
54
54
54


In [38]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans with safety checks.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    # Safe access for decoder_start_token_id
    decoder_start_token_id = getattr(model.config, "decoder_start_token_id", None)
    pad_token_id = getattr(tokenizer, "pad_token_id", None)
    eos_token_id = getattr(tokenizer, "eos_token_id", None)

    if decoder_start_token_id is None:
        print("[Warning] decoder_start_token_id is None. Using default generation behavior.")

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]
        print('here')

        # Preprocess the batch
        batch_preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        if not isinstance(batch_preprocessed, list) or len(batch_preprocessed) == 0:
            print(f"[Warning] Preprocessed batch is empty at index {i}. Skipping...")
            continue

#         Debug: print first 2 sentences after preprocessing
        print(f"[Debug] Preprocessed batch sample: {batch_preprocessed[:2]}")

        # Tokenize
        inputs = tokenizer(
            batch_preprocessed,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations with safety parameters
        with torch.no_grad():
            try:
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=5,
                    max_length=max_length,
                    num_beams=5,
                    num_return_sequences=1,
                    early_stopping=True,
                    decoder_start_token_id=decoder_start_token_id,
                    pad_token_id=pad_token_id,
                    eos_token_id=eos_token_id
                )
            except Exception as e:
                print(f"[Error] Generation failed for batch starting at index {i}: {e}")
                continue

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

#         Debug: print first 2 decoded outputs
        print(f"[Debug] Decoded sample: {decoded_texts[:2]}")

        # Postprocess translations
        try:
            postprocessed = ip.postprocess_batch(decoded_texts, lang=tgt_lang)
            translations += postprocessed
        except Exception as e:
            print(f"[Error] Postprocessing failed for batch starting at index {i}: {e}")
            translations += decoded_texts  # fallback

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations

In [39]:
src_lang, tgt_lang = "eng_Latn", "mar_Deva"

prefix = f"{tgt_lang} {src_lang}"

def remove_prefix(text):
    if text.startswith(prefix):
        return text[len(prefix):].strip()
    return text.strip()

In [40]:
from evaluate import load
from tqdm import tqdm
import pandas as pd

# -------------------- TRANSLATION --------------------
valid_src = []
valid_pred = []
valid_gt = []
valid_gem = []
valid_cfilt = []

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
#     print(batch)
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        en_indic_model,
        en_indic_tokenizer,
        ip,
        device=DEVICE
    )
    all_translations.extend(translations)

    if translations is None:
        print(f"[SKIPPED] Batch {i}: Returned None")
        continue

    cleaned = [remove_prefix(t) for t in translations]

    valid_src.extend(batch)
    valid_pred.extend(cleaned)
    valid_gt.extend(ref_gt[i:i + len(batch)])
    valid_gem.extend(ref_gem[i:i + len(batch)])
    valid_cfilt.extend(ref_cfilt[i:i + len(batch)])

print(f"\nSuccessful translations: {len(valid_pred)} / {len(src_sentences)}")

  0%|                                                | 0/14 [00:00<?, ?it/s]

here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Chanting : the choir raised the volume as the celebrant intoned the prayer .', 'eng_Latn mar_Deva A six-month-old calf was submitted for examination , showing lameness in all four legs , which had been present since soon after birth .']


  7%|██▊                                     | 1/14 [00:00<00:11,  1.15it/s]

[Debug] Decoded sample: ['जपः गायकवृंदाने आवाज वाढवला आणि उत्सवी व्यक्तीने प्रार्थना केली.', 'जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणाऱ्या सहा महिन्यांच्या वासराला तपासणीसाठी सादर करण्यात आले.']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva As mentioned , , first impressions can be misleading .', 'eng_Latn mar_Deva To get a clean assembly load , the assembled equals table before the assembly is run .']


 14%|█████▋                                  | 2/14 [00:01<00:08,  1.48it/s]

...

[Debug] Decoded sample: ['पी. टी. टी. - पुरविलेल्या पॅकेट - स्विच नेटवर्कशी जोडणी ही वर्कस्टेशनची प्रमुख आवश्यकता असेल आणि अखंड डेटा संप्रेषण सुनिश्चित करण्यासाठी पी. टी. टी. द्वारे या नेटवर्कमधील प्रवेशद्वारांचे नियोजन केले जाते.', 'या प्रणालीने सातत्याने 50 गॅलन प्रति तास इष्टतम उत्पादन साध्य केले.']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Morphological studies of paraquat or oxygen toxicity in rats have shown significant cellular damage in lung tissue .', 'eng_Latn mar_Deva One should read g . v . carey \' " s chapter " proof correction in " mind the stop .']


 93%|████████████████████████████████████▏  | 13/14 [00:07<00:00,  1.71it/s]

[Debug] Decoded sample: ['उंदरांमधील पॅराक्वाट किंवा ऑक्सिजन विषाक्ततेच्या आकारशास्त्रीय अभ्यासात फुफ्फुसाच्या ऊतींमध्ये लक्षणीय पेशी नुकसान झाल्याचे दिसून आले आहे.', "जी. व्ही. कॅरीचा'प्रूफ करेक्शन'हा अध्याय'माइंड द स्टॉप'मध्ये वाचला पाहिजे."]
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva A water : - glycol ratio of < ID1 > is preferred .', 'eng_Latn mar_Deva It has thickness of only 16 , : 240 inch .']


100%|███████████████████████████████████████| 14/14 [00:08<00:00,  1.74it/s]

[Debug] Decoded sample: ['पाणीः - ग्लायकोल गुणोत्तर < आयडी1 > ला प्राधान्य दिले जाते.', 'त्याची जाडी केवळ 16,240 इंच आहे.']

Successful translations: 54 / 54


In [41]:
mode = "approach1_bert"

In [42]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": valid_src,
    "prediction": valid_pred,
    "gt": valid_gt,
    "gemini": valid_gem,
    "cfilt": valid_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(valid_pred, valid_gt, valid_gem, valid_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gt])["score"],
    "Gemini": bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gem])["score"],
    "CFILT":  bleu.compute(predictions=valid_pred, references=[[r] for r in valid_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to approach1_bert_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 75.95, chrF++: 87.50
GT Marathi → BLEU: 59.73
Gemini    → BLEU: 23.84
CFILT     → BLEU: 59.91

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_approach1_bert_baseline_outputs_eval_metrics.txt


In [51]:
results_df['prediction'][0]

'अफगाणिस्तानमध्ये हिंदू खूप कमी आहेत आणि मालदीवमध्ये हिंदू नाहीत.'

In [34]:
src_sentences = raw_datasets['test']["sent_meant"]

In [35]:
from evaluate import load
from tqdm import tqdm
import pandas as pd

# -------------------- TRANSLATION --------------------
valid_src = []
valid_pred = []
valid_gt = []
valid_gem = []
valid_cfilt = []

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
#     print(batch)
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        en_indic_model,
        en_indic_tokenizer,
        ip,
        device=DEVICE
    )
    all_translations.extend(translations)

    if translations is None:
        print(f"[SKIPPED] Batch {i}: Returned None")
        continue

    cleaned = [remove_prefix(t) for t in translations]

    valid_src.extend(batch)
    valid_pred.extend(cleaned)
    valid_gt.extend(ref_gt[i:i + len(batch)])
    valid_gem.extend(ref_gem[i:i + len(batch)])
    valid_cfilt.extend(ref_cfilt[i:i + len(batch)])

print(f"\nSuccessful translations: {len(valid_pred)} / {len(src_sentences)}")

  0%|                                                | 0/14 [00:00<?, ?it/s]

here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Chanting , the choir raised the volume as the celebrant intoned the prayer .', 'eng_Latn mar_Deva A six-month-old calf was submitted for examination , showing lameness in all four legs , which had been present since soon after birth .']


  7%|██▊                                     | 1/14 [00:00<00:11,  1.16it/s]

[Debug] Decoded sample: ['उत्सवी व्यक्तीने प्रार्थनेचा उच्चार करत असताना, गायकवृंदाने जप करत आवाज वाढवला.', 'जन्मानंतर लगेचच अस्तित्वात असलेल्या चारही पायांमध्ये लंगडेपणा दर्शविणाऱ्या सहा महिन्यांच्या वासराला तपासणीसाठी सादर करण्यात आले.']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva As mentioned first , impressions can be misleading .', 'eng_Latn mar_Deva To get a clean assembly , load the assembled equals table before the assembly is run .']


 14%|█████▋                                  | 2/14 [00:01<00:10,  1.18it/s]

...

[Debug] Decoded sample: ['पी. टी. टी. - पुरविलेल्या पॅकेट - स्विच नेटवर्कशी जोडणी ही वर्कस्टेशनची प्रमुख आवश्यकता असेल आणि अखंड डेटा संप्रेषण सुनिश्चित करण्यासाठी पी. टी. टी. द्वारे या नेटवर्कमधील प्रवेशद्वारांचे नियोजन केले जाते.', 'प्रणालीने सातत्याने 50 गॅलन प्रति तास साध्य केले - इष्टतम उत्पादन.']
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva Morphological studies of paraquat- or oxygen-toxicity in rats have shown significant cellular damage in lung tissue .', "eng_Latn mar_Deva One should read G. V. Carey 's chapter ' Proof Correction ' in Mind the Stop ."]


 93%|████████████████████████████████████▏  | 13/14 [00:11<00:01,  1.00s/it]

[Debug] Decoded sample: ['उंदरांमधील पॅराक्वाट - किंवा ऑक्सिजन - विषाक्ततेच्या आकारशास्त्रीय अभ्यासातून फुफ्फुसाच्या ऊतींमध्ये लक्षणीय पेशीय नुकसान झाल्याचे दिसून आले आहे.', "जी. व्ही. कॅरी यांचा'प्रूफ करेक्शन'हा अध्याय'माइंड द स्टॉप'मध्ये वाचला पाहिजे."]
here
[Debug] Preprocessed batch sample: ['eng_Latn mar_Deva a water / glycol ratio of < ID1 > is preferred .', 'eng_Latn mar_Deva It has thickness of only < ID1 > inch .']


100%|███████████████████████████████████████| 14/14 [00:12<00:00,  1.12it/s]

[Debug] Decoded sample: ['< आयडी1 > चे पाणी / ग्लायकोल गुणोत्तर पसंत केले जाते.', 'त्याची जाडी फक्त < आयडी1 > इंच आहे.']

Successful translations: 54 / 54


In [36]:
mode = "sent_meant"

In [37]:
import pandas as pd
from evaluate import load

# -------------------- SAVE OUTPUTS --------------------
results_df = pd.DataFrame({
    "src": valid_src,
    "prediction": valid_pred,
    "gt": valid_gt,
    "gemini": valid_gem,
    "cfilt": valid_cfilt
})

results_df.to_csv(f"{mode}_outputs.csv", index=False)
print(f"✔ Saved predictions to {mode}_outputs.csv")

# -------------------- METRICS --------------------
bleu = load("sacrebleu")
chrf = load("chrf")

def compute_scores(preds, ref1, ref2, ref3):
    """
    Compute BLEU and chrF++ scores using all three references for each sentence.
    """
    references = [[r1, r2, r3] for r1, r2, r3 in zip(ref1, ref2, ref3)]  # sacrebleu format
    bleu_score = bleu.compute(predictions=preds, references=references)["score"]
    chrf_score = chrf.compute(predictions=preds, references=references)["score"]
    return bleu_score, chrf_score

bleu_score, chrf_score = compute_scores(valid_pred, valid_gt, valid_gem, valid_cfilt)

# Determine best reference per metric (based on BLEU)
all_scores = {
    "GT":    bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gt])["score"],
    "Gemini": bleu.compute(predictions=valid_pred, references=[[r] for r in valid_gem])["score"],
    "CFILT":  bleu.compute(predictions=valid_pred, references=[[r] for r in valid_cfilt])["score"]
}

best_ref = max(all_scores, key=all_scores.get)

print("\n===== FINAL METRICS =====")
print(f"All references combined → BLEU: {bleu_score:.2f}, chrF++: {chrf_score:.2f}")
print(f"GT Marathi → BLEU: {all_scores['GT']:.2f}")
print(f"Gemini    → BLEU: {all_scores['Gemini']:.2f}")
print(f"CFILT     → BLEU: {all_scores['CFILT']:.2f}")
print(f"\n🎯 BEST REFERENCE = {best_ref} (by highest BLEU)")

# -------------------- SAVE METRICS --------------------
with open(f"punct_{mode}_indictrans2_eval_metrics.txt", "w") as f:
    f.write(f"All references combined → BLEU {bleu_score:.2f}, chrF++ {chrf_score:.2f}\n")
    f.write(f"GT    BLEU {all_scores['GT']:.2f}\n")
    f.write(f"Gem   BLEU {all_scores['Gemini']:.2f}\n")
    f.write(f"CFILT BLEU {all_scores['CFILT']:.2f}\n")
    f.write(f"\nBEST REFERENCE = {best_ref}\n")

print(f"Metrics written to punct_{mode}_baseline_outputs_eval_metrics.txt")

✔ Saved predictions to sent_meant_outputs.csv

===== FINAL METRICS =====
All references combined → BLEU: 83.34, chrF++: 90.50
GT Marathi → BLEU: 63.29
Gemini    → BLEU: 26.20
CFILT     → BLEU: 67.17

🎯 BEST REFERENCE = CFILT (by highest BLEU)
Metrics written to punct_sent_meant_baseline_outputs_eval_metrics.txt


In [40]:
predicted_sentences[7:13]

['These glycans are poorly transferred to proteins resulting in unoccupied, glycosylation sequons.',
 'X is an effective acute, oral treatment for migraine with a rapid onset of action',
 'No newspaper is completely unbiased in, my, expert, opinion.',
 'Steam, for, example is, just as damaging as acid for that material.',
 'My favourite opera composers are Verdi, Puccini, Mozart and Gilbert and Sullivan.',
 'The only problem with the new project established in the desert at high cost is the lack of good access roads.']

In [41]:
src_sentences[7:13]

['These glycans are poorly transferred to proteins, resulting in unoccupied glycosylation sequons.',
 'X is an effective acute, oral treatment for migraine, with a rapid onset of action',
 'No newspaper is completely unbiased, in my expert opinion.',
 'Steam, for example, is just as damaging as acid for that material.',
 'My favourite opera composers are Verdi, Puccini, Mozart, and Gilbert and Sullivan.',
 'The only problem with the new project, established in the desert at high cost, is the lack of good access roads.']